<font size=8>Graph Page Source Code</font>

# <font color="gray">Set Up Notebook</font>

This section gets everythng ready for us to start coding. Specifically it imports all of the required packages and loads in the datasets we need of the website.

## Import Packages

Shiny and stats packages

In [ ]:
require(shiny)
require(digest) 
require(grid)
require(MASS) 
require(stats) 
require(shinyjs) 

Plotting Packages

In [ ]:
library(ggplot2)
library(plotly)
library(ggthemes)

## Download Everything <small>(if necessary)</small>

This downloads the immunological data files that are used to create the graphs 

In [ ]:
if( !'data' %in% list.files() ){
    print("Downloading data files")
    dir.create("data")
    download.file(url="https://storage.googleapis.com/bakar-data/10k/data_app/newtenkdata.rdata",
                  destfile="data/newtenkdata.rdata", method="wget") # Kelly's 10k dataset
    download.file(url="https://storage.googleapis.com/bakar-data/10k/data_app/Guinea_RNA.csv",
                  destfile="data/Guinea_RNA.csv", method="wget") # New Guinea RNA data
    download.file(url="https://storage.googleapis.com/bakar-data/10k/data_app/png_metabolomics.725331.csv",
                  destfile="data/png_metabolomics.725331.csv", method="wget") # New Guinea Metabolomics dataset
    download.file(url="https://storage.googleapis.com/bakar-data/10k/data_app/guinea_cibersort.csv",
                  destfile="data/guinea_cibersort.csv", method="wget") # New Guinea Metabolomics dataset
    print("Done")
}

## Load in Datasets and Resources

 Add the `data` folder to R Shiny's file path. Se that we can display images in the website.

In [ ]:
addResourcePath("data", paste(getwd() , "/data", sep="") )

load in a and format `guinea_cibersort` data

In [ ]:
guinea_cibersort = read.csv( "data/guinea_cibersort.csv",  stringsAsFactors=FALSE, header=TRUE, sep="," )
guinea_cibersort$DAY =  as.numeric( substr( guinea_cibersort$Input.Sample, 6,6 ) )
guinea_cibersort$SUBJECT = substr( guinea_cibersort$Input.Sample, 1,3 ) 

Load in `guinea_mass_spec` data

In [ ]:
guinea_mass_spec = read.csv( "data/png_metabolomics.725331.csv",  stringsAsFactors=FALSE, header=TRUE, sep="," )
guinea_mass_spec[guinea_mass_spec=="M"] = "Male"
guinea_mass_spec[guinea_mass_spec=="F"] = "Female"
guinea_mass_spec[guinea_mass_spec=="DelayedGroup"] = "Delayed Group"
guinea_mass_spec[guinea_mass_spec=="VaccinatedGroup"] = "Vaccinated Group"

#  <font color="blue">Initialize Shiny</font> 

To make this projcect more readable we I user **<font color="green">user interface</font>** and **<font color="purple">server</font>** object. I then iteratively append code to these objects a few pieces at a time.

This line creates the **<font color="green">user interface</font>**, the "front end" part of the website that everyone sees.

In [ ]:
ui = div() 

We create the **<font color="purple">server</font>** object then we add code to it one at a time in the followign sections.

In [ ]:
server <- function(input, output, session) { }

## <font color="gray">Helper Function: callConcat</font>

<font color="blue">callConcat</font> is a function that combines R expressions. [Link to solution](https://codeday.me/en/qa/20190306/8275.html). I use it to append code to the **<font color="purple">Server Object</font>**

In [ ]:
callConcat = function(...) {
    ll <- list(...)
    ll <- lapply(ll, function(X) as.list(X)[-1])
    ll <- do.call("c", ll)
    as.call(c(as.symbol("{"), ll))
}

#  <font color="orange">Titles</font>

This is the codes that displays the website material at the top of the page. It's all the code that is above the main buttons of the website. 

we make the title **page template**

In [ ]:
tempHtml= div( class="row",
       div( class="jumbotron", style="padding: .9em; background-size: cover ",
       div( style="display: flex;justify-content: center; position: relative; ",
           h1(style="text-align: center;", id = "graph_page_header", "Graph title text" ),
           actionButton("homeBtn", icon = icon("home"), label="", class="btn btn-link", style="  outline: none; padding: 15px; border: 0px; background-color: transparent; font-size: 260%;    ") 
       ), 
       p( tags$small( id="graph_page_text", style="text-align:center;", "Graph Body Text"))
  ))

ui = tagAppendChild( ui , tempHtml )
#tempHtml


# <font color="SkyBlue">Buttons</font>

This code creates the <font color="SkyBlue">Buttons</font> used to switch between webpages.

In [ ]:
tempHtml= div(
    column(3, actionButton("geneBtn", icon = icon("dna"), label="Fake Button", class="btn btn-success btn-lg text-center", style="width: 100%"), style="padding: 2px; "),
    column(3, actionButton("massSpecBtn", icon = icon("cogs"), label="Fake", class="btn-primary btn-lg text-center",style="width: 100%"), style="padding: 2px;"),
    column(3, actionButton("antibodyBtn", icon = icon("yahoo"), label="Fake", class="btn btn-warning btn-lg text-center", style="width: 100%"), style="padding: 2px;"),
    column(3, actionButton("labBtn", icon = icon("vial"), label="Fake", class="btn btn-danger btn-lg text-center", style="width: 100%"),  style="padding: 2px; "),
    p(".", style="color:white;"),
    HTML("<hr>")
)

ui = tagAppendChild(ui, tempHtml )
#tempHtml


# <font color="peru">Body</font>

This code displays everything on the website that is below the <font color="SkyBlue">Buttons</font>. To get a better idea of what it does  uncomment the `tempHtml` line at the end of the  <font color="peru">Body Home</font> section

The code below crete the <font color="peru">Page Body</font>. The is broken up into to basic sections, the **sidebar** and the **main panel**.
* **sidebar:** This is where all the widgets are that control what is graphed
* **main panel:** This small piece of code displays the graph

## <font color="peru">Sidebar

In [ ]:
sidebar = sidebarPanel( 
        
        #Select which data type to visualize
        h4('Select Data Type:'),
        selectInput('dataType', label = NULL,
                    choices = c( 'RNAseq: T cells', 'RNAseq: Newborns', 'Mass Spectrometry: Newborns', 'Cibersort: Newborns'),
                    selected = 'Cibersort: Newborns'
        ),
        # Analyte Selection Box
         h4('Select Analyte:'),
         selectInput('cibersort_analyte', label = NULL,
                     choices = colnames( guinea_cibersort )[2:23],
                     selected = 'Neutrophils' ),
        # Plot By Box
         h4('Plot By:'),
         radioButtons( 'newbornPlotBy', label = NULL,
                      choices = c('Days of Life','Sex','Vaccination'),
                      selected = 'Days of Life'
         )
)

## <font color="gray">Helper</font> <font color="blue">myDownloadButton</font>

I customized R Shiny's download button to make it look nice for the app. The code I used to do this comes [from here](https://stackoverflow.com/questions/49350509/adding-removing-icon-in-downloadbutton-and-fileinput)

In [ ]:
myDownloadButton <- function(outputId, label = "Download", myIcon="download", class = NULL, ... ){
     tags$a(id = outputId, class = paste("btn btn-default shiny-download-link", class),
         href = "", target = "_blank", download = NA, 
         icon(myIcon) , label, ...)
}

## <font color="peru">Graph

<font color="red">Note:</font> in the future uncomment out `numSubjects` to print number of subjects.

In [ ]:
main_panel = mainPanel(  
        fluidRow( div( class="col-md-offset-9  col-xs-offset-8",  #col-sm-offset-8 col-lg-offset-9 
                   tags$b( style="font-family: 'Times New Roman'; color: DarkSlateGrey; font-size: 115%", textOutput("num_subjects",inline = F) ) # 
                 ) ),
        plotlyOutput("dataPlot"),
        h2( class="col-xs-offset-2 col-md-offset-1", "Download"),
        div(
            myDownloadButton('downloadPlot', label='Image', myIcon="image", class="btn btn-success"),
            myDownloadButton('downloadPlotData', label='Plot Data', myIcon="file-download", class="btn btn-warning")
        ),
        p( style = 'color: DarkGrey; padding: 0px;', "* All Data is dataset's raw and formatted files")
      )



Putting Graph and SideBar together

In [ ]:
tempHtml = sidebarLayout( sidebar, main_panel )             
ui = tagAppendChild(ui, tempHtml )
#tempHtml

## <font color="peru">Data Explanations

Template for Data explanation text

In [ ]:
tempHtml = div( class="jumbotron", style="background-color: white; padding: .9em;",  # padding-left: .9em; padding-top: 0px;",
         p( id="graph_data_info" , tags$b("Data Info: "), tags$small("Put data info here" ) )
   )                        

ui = tagAppendChild(ui, tempHtml )


# <font color="purple">Graph Functions</font>

`elliottData` creates the dataset used to make make the plot

In [ ]:
temp= quote({
    elliottData <- reactive({
        
        # get cibersort analyte and create graph dataframe
        col_num = which( colnames( guinea_cibersort ) == input$cibersort_analyte )[1]
        to_graph= data.frame( Expression=  round(guinea_cibersort[,col_num],3) , Group= as.factor(guinea_cibersort$DAY) )

        # add additional columns to to_graph
        to_graph$Day = guinea_cibersort$DAY #+ rnorm( dim(guinea_cibersort)[1] , mean = 0, sd = .12 ) 
        to_graph$DayPlot = guinea_cibersort$DAY + rnorm( dim(guinea_cibersort)[1] , mean = 0, sd = .15 ) 
        to_graph$Vaccination = guinea_mass_spec[ match( guinea_cibersort$SUBJECT, guinea_mass_spec$SUBJECT), 5 ]
        to_graph$Sex = guinea_mass_spec[ match( guinea_cibersort$SUBJECT, guinea_mass_spec$SUBJECT), 6 ]
        to_graph$Subject = guinea_cibersort$SUBJECT

        to_graph # <- return dataset
    })
})
body(server) = callConcat( body(server), temp )


We write the funciton `elliottPlot` that creates the plot

In [ ]:
temp= quote({
    elliottPlot <- reactive({
        
        to_graph = elliottData()  # <- Get dataset
        
        # Decide how to plot data based on "newbornPlotBy" input value
        main_plot = switch( input$newbornPlotBy ,
             'Days of Life' = ggplot( to_graph, aes(x=DayPlot, day=Day, y=Expression, color=Group, sex=Sex, status=Vaccination, subject=Subject ) ),
             'Sex'= ggplot( to_graph, aes(x=DayPlot, day=Day, y=Expression, color=Sex, status=Vaccination, subject=Subject ) ),
             'Vaccination' = ggplot( to_graph, aes(x=DayPlot, day=Day, y=Expression, color=Vaccination, sex=Sex, subject=Subject ) ) 
             ) 
        
        my_ylab = switch( input$dataType ,
              'RNAseq: Newborns' = "Gene Expression (counts)",
              "Mass Spectrometry: Newborns" = "Molecule Expression",
              "Phenotype Expression"
              )
     
        p = main_plot + geom_point(alpha=.7) + theme_gdocs() + labs(x='Days Alive', y=my_ylab, title=paste( input$cibersort_analyte," Expression in Newborns") ) 
        ggplotly( p , tooltip = c("Day", "Expression", "Subject", "Sex", "Vaccination" ) )
    })
})
body(server) = callConcat( body(server), temp )


**<font color="purple">Render  Plot**

In [ ]:
temp= quote({
    plotInput <- reactive({
        p <- elliottPlot()
        my_data = elliottData()
        output$num_subjects = renderText({  paste( dim(my_data)[1]," Subjects" )  })
        p
    }) #<-- plotInput end    
    output$dataPlot <- renderPlotly({ plotInput() })
    
})  #<-- Quote end
body(server) = callConcat( body(server), temp )


# <font color="blue">Run  Shiny</font> 

We change the following options so that <font color="blue">RShiny</font> runs at an exposed port and on the computer.

In [ ]:
options(shiny.port = 8888)
options(shiny.host = "0.0.0.0")

We run the app!

In [ ]:
shinyApp(ui = fluidPage( useShinyjs(), style='margin-left:5px; margin-right:5px', ui  ), server = server)